In [121]:
#El plan es que en las coordenadas con agua ponemos '0', en las coordenadas con barcos ponemos '1'
# y en las coordenadas donde se ha disparado ponemos 2, al pesar que originalmente había un '0' o '1'

#self.barcos es un diccionario con llaves la dimension del barco y valor cuanto barcos te quedan

#set_boats: select which boat from the dictionary barcos, select a random orientation (horizontal or vertical), then 
# set the boat at random in the allowed area, considering that position is the head of the boat
#  (the rest of the boat follows either right if orientation==horizontal or down if orientation==vertical)

#the allowed area have zeros, I set with '2' the area close to the boats already set before, so that they don't touch.
#When all the boats are set, I will reset all the '2' with '0'

#barcos = {1 : 4, 2 : 3 , 3 : 2 , 4 : 1}


#set_mask
# I pad the matrix with an extra border, not to be worried if the boat is at the border or not. I set the mask and then I 
# delete the extra padding I put.

#CONSTANTS:
#BOARD_SIZE = (10,10)

import numpy as np

class Board:

    
    def __init__(self, player, board_size, boats):
        self.player = player
        self.board = np.zeros(board_size)
        self.xdim = board_size[0]
        self.ydim = board_size[1]
        self.boats = boats

        for boat, num in boats.items():
            for j in range(num):
                self.set_boat(boat)
                
        self.polish_board()


    def set_boat(self, dim_boat):
        flag = True
        if dim_boat not in self.boats:
            print(f"ERROR: There is no boat with dimension {dim_boat}")
            return None
        if self.boats[dim_boat]>0:
            self.boats[dim_boat]-=1 #substract the boat you already put
        else:
            print(f"Warning: you already put all the boats of dimension {dim_boat}")
            return None
        
        orientation = np.random.randint(0,2)
        while flag:
            if orientation ==0: # horizontal
                coord1 = np.array([np.random.randint(0,self.xdim),np.random.randint(0,self.ydim-dim_boat+1)])
                coord2 = np.array([coord1[0], coord1[1] + dim_boat])

                #if all the points where I want to put the boat are free (zeros)
                if (self.board[coord1[0]:coord2[0]+1,coord1[1]:coord2[1]] == 0).all():
                    self.board[coord1[0]:coord2[0]+1,coord1[1]:coord2[1]] = 1
                    self.set_mask(coord1, coord2, orientation)
                    flag = False
            elif orientation ==1:#vertical
                coord1 = np.array([np.random.randint(0,self.xdim-dim_boat+1),np.random.randint(0,self.ydim)])
                coord2 = np.array([coord1[0] + dim_boat, coord1[1]])

                
                if (self.board[coord1[0]:coord2[0],coord1[1]:coord2[1]+1]==0).all():
                    self.board[coord1[0]:coord2[0],coord1[1]:coord2[1]+1] = 1
                    self.set_mask(coord1, coord2, orientation)
                    flag = False


    def set_mask(self, coord1, coord2, orientation):
        #pad the matrix:
        new_board = np.zeros((self.xdim+2, self.ydim+2))
        new_board[1:-1,1:-1] = self.board
        #add one to coordinates
        coord1 +=1
        coord2 +=1

        if orientation==0: #horizontal
            new_board[coord1[0],coord1[1]-1] = 2
            new_board[coord2[0],coord2[1]] = 2
            new_board[coord1[0]-1, coord1[1]:coord2[1]] = 2
            new_board[coord1[0]+1, coord1[1]:coord2[1]] = 2
        else:
            new_board[coord1[0]-1,coord1[1]] = 2
            new_board[coord2[0],coord1[1]] = 2
            new_board[coord1[0]:coord2[0], coord1[1]-1] = 2
            new_board[coord1[0]:coord2[0], coord1[1]+1] = 2

        #delete padding
        self.board = new_board[1:-1,1:-1]
              
    def polish_board(self):
        self.board[np.where(self.board==2)]=0
        print(f"The board for player {self.player} is ready")



In [126]:
boats = {1 : 4, 2 : 3 , 3 : 2 , 4 : 1}

my_board = Board("Fabrizio",(10,10),boats.copy())

print(boats)
print(my_board.board)
print("\n\n")

cpu_board = Board("Computer",(10,10),boats.copy())

print(cpu_board.board)


The board for player Fabrizio is ready
{1: 4, 2: 3, 3: 2, 4: 1}
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 1.]]



The board for player Computer is ready
[[0. 1. 0. 1. 0. 1. 1. 0. 0. 1.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
boats = {1 : 4, 2 : 3 , 3 : 2 , 4 : 1}




In [75]:
coord1 = np.array([np.random.randint(0,10),np.random.randint(0,10)])
print(coord1)
coord1 +=1
print(coord1)

[8 1]
[9 2]


In [90]:
matrix = np.random.randint(0,10,(2,5))

In [91]:
matrix

array([[4, 8, 3, 9, 0],
       [0, 4, 5, 4, 7]])

In [ ]:
matrix[1]